In [1]:
from datetime import datetime, timedelta
from pyspark.sql import functions

In [2]:
num_days = 1
days = [datetime.utcnow().date() - timedelta(1) - timedelta(i) for i in range(0, num_days)]

In [3]:
dataset = SQLContext(sc).read.load(['s3://telemetry-parquet/socorro_crash/v1/crash_date=' + day.strftime('%Y%m%d') for day in days], 'parquet')

In [4]:
modules = dataset\
.filter(dataset['product'] == 'Firefox')\
.select(functions.explode(dataset['json_dump']['modules']).alias('module')).rdd\
.map(lambda v: v['module'])\
.filter(lambda m: m['missing_symbols'])\
.flatMap(lambda m: [((m['filename'], m['version']), 1)])\
.reduceByKey(lambda x, y: x + y)\
.sortBy(lambda v: v[1], ascending=False)\
.collect()

In [5]:
len(modules)

22785

In [7]:
modules[:30]

[((u'ntdll.dll', u'6.1.7601.23569'), 35215),
 ((u'ntdll.dll', u'10.0.14393.447'), 32694),
 ((u'KERNELBASE.dll', u'10.0.14393.321'), 30611),
 ((u'user32.dll', u'10.0.14393.351'), 25864),
 ((u'kernel32.dll', u'6.1.7601.23569'), 20858),
 ((u'msmpeg2vdec.dll', u'12.0.9200.17037'), 13971),
 ((u'ntdll.dll', u'6.1.7601.23543'), 13490),
 ((u'combase.dll', u'10.0.14393.351'), 12682),
 ((u'KERNELBASE.dll', u'6.1.7601.23569'), 12610),
 ((u'IPSEng32.dll', u'15.2.4.5'), 9453),
 ((u'kernel32.dll', u'6.1.7601.23543'), 8034),
 ((u'msmpeg2vdec.dll', u'10.0.14393.187'), 6464),
 ((u'inproc_agent.dll', u'4.6.1.201'), 4985),
 ((u'IPSEng32.dll', u'15.2.3.14'), 4940),
 ((u'libsystem_kernel.dylib', u''), 4761),
 ((u'atidxx32.dll', u'8.17.10.625'), 4694),
 ((u'jvm.dll', u'24.51.0.3'), 4573),
 ((u'npjp2.dll', u'10.51.2.13'), 4570),
 ((u'KERNELBASE.dll', u'6.1.7601.23543'), 4552),
 ((u'rooksbas.dll', u'3.2.0.1'), 4134),
 ((u'ntdll.dll', u'10.0.10586.672'), 3723),
 ((u'LavasoftTcpService.dll', u'2.3.4.7'), 3637),